In [62]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('Data/raw_residential_data.csv')
df2 = pd.read_csv('Data/raw_address_points.csv')

C:\Users\X\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The next three cells create a smaller data frame with useable prices.
- 1) First create a Smaller DF containing only records with prices since 2010.
- 2) Then index the prices to smooth out secular and seasonal trends in prices.  Using an index created from monthly medians. This is done by creating a df of monthly indeces.
- 3) Join the new df to the one from 1) above, and create an indexed price.  This price should be usable for analysis.     

In [63]:
# Creating a smaller DF that has only records with prices with sales in 2010 and beyond.
df1['SALEDATE']=pd.to_datetime(df1['SALEDATE']) # first convert the Saledate to a datetime format
df1['YearMonth'] = df1['SALEDATE'].map(lambda x: 100*x.year + x.month) # Create a "Year/Month Field"

# Now create the new, smaller df
dfPrice = df1[df1.PRICE > 0]
dfPrice = dfPrice[dfPrice.YearMonth >= 201001]

# Now find the median of all records to use as a base for the index.
MedianIndexPrice = dfPrice.PRICE.median()

print('Number of Records ',len(dfPrice), '-- Median Price = ',MedianIndexPrice)

Number of Records  31158 -- Median Price =  606375.0


In [64]:
# Now create a df with median price by month
MonthlyMedians = dfPrice.groupby(['YearMonth'])['PRICE'].median()
Monthlydf = pd.DataFrame(MonthlyMedians) # turn it into a data frame
Monthlydf['MonthIndex']=Monthlydf.PRICE/MedianIndexPrice # Create an index for the month
Monthlydf=Monthlydf.rename(columns = {'PRICE':'MonthlyMedian'}) # Rename "PRICE" Column to describe it and avoid confusion

Monthlydf.head()

,MonthlyMedian,MonthIndex
YearMonth,,
201001,390000.0,0.643166
201002,399500.0,0.658833
201003,382500.0,0.630798
201004,395400.0,0.652072
201005,410000.0,0.676149


In [65]:
df = pd.merge(dfPrice,Monthlydf,on='YearMonth') #Combine the two dfs
df['IndexedPrice'] = df.PRICE/df.MonthIndex
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31158 entries, 0 to 31157
Data columns (total 43 columns):
OBJECTID             31158 non-null int64
SSL                  31158 non-null object
BATHRM               31158 non-null float64
HF_BATHRM            31157 non-null float64
HEAT                 31158 non-null float64
HEAT_D               31158 non-null object
AC                   31158 non-null object
NUM_UNITS            31158 non-null float64
ROOMS                31151 non-null float64
BEDRM                31156 non-null float64
AYB                  31153 non-null float64
YR_RMDL              20485 non-null float64
EYB                  31158 non-null int64
STORIES              31125 non-null float64
SALEDATE             31158 non-null datetime64[ns]
PRICE                31158 non-null float64
QUALIFIED            31158 non-null object
SALE_NUM             31158 non-null int64
GBA                  31158 non-null int64
BLDG_NUM             31158 non-null int64
STYLE             

In [66]:
df.groupby(['YearMonth'])['IndexedPrice','PRICE'].median()

,IndexedPrice,PRICE
YearMonth,,
201001,606375.0,390000.0
201002,606375.0,399500.0
201003,606375.0,382500.0
201004,606375.0,395400.0
201005,606375.0,410000.0
201006,606375.0,450000.0
201007,606375.0,502500.0
201008,606375.0,549450.0
201009,606375.0,380000.0
